<a href="https://colab.research.google.com/github/look4pritam/GenerativeAdversarialNetworks/blob/master/Notebooks/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Generative Adversarial Networks

In this assignment, we will learn to generate images using [Deep Convolutional Generative Adversarial Networks](https://en.wikipedia.org/wiki/Generative_adversarial_network) (GAN).

We will use a Convolutional Neural Network based DCGAN for image generation.

See [link](https://en.wikipedia.org/wiki/Generative_adversarial_network) for more details.

# Set the root directory for processing.

In [1]:
import os

root_dir = '/content/'
os.chdir(root_dir)

!ls -al

total 16
drwxr-xr-x 1 root root 4096 Mar 17 13:32 .
drwxr-xr-x 1 root root 4096 Mar 19 06:19 ..
drwxr-xr-x 4 root root 4096 Mar 17 13:31 .config
drwxr-xr-x 1 root root 4096 Mar 17 13:32 sample_data


# Import required python modules.

In [2]:
import numpy as np
np.random.seed(7)

In [3]:
import tensorflow as tf
tf.random.set_seed(7)

import tensorflow_datasets as tfds

# Define input image shape, batch size, and buffer size.

In [4]:
image_shape = (28, 28, 1)
batch_size = 64
buffer_size = 1024

# Load MNIST dataset using TensorFlow dataset.

### Define a dataset augmentation function.

In [5]:
def augment_dataset(image, label):
  return (image, label)

### Define a function to load the training dataset.

In [6]:
def load_train_dataset(batch_size, buffer_size):
  number_of_batches = 0
  train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)
  train_dataset = train_dataset.concatenate(test_dataset)

  train_dataset = train_dataset.shuffle(buffer_size)
  train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
  train_dataset = train_dataset.map(augment_dataset)

  return (train_dataset, number_of_batches)

### Define a function to normalize the dataset.

In [7]:
def normalize_dataset(image, label):
  image = (tf.cast(image, tf.float32) - 127.5) / 127.5
  return (image, label)

### Define a function to preprocess the training dataset.

In [8]:
def preprocess_train_dataset(batch_size, buffer_size):
  train_dataset, number_of_batches = load_train_dataset(batch_size, buffer_size)
  train_dataset = train_dataset.map(normalize_dataset)
  return (train_dataset, number_of_batches)

### Preprocess the training dataset.

In [9]:
train_dataset, number_of_batches = preprocess_train_dataset(batch_size, buffer_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.4X3B1Y_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.4X3B1Y_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


# Import required python modules.

In [10]:
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

In [12]:
from tensorflow.keras.layers import BatchNormalization

# Define a latent dimension.

In [11]:
latent_dimension = 128

In [ ]:
"""
# Define a customized BatchNormalization layer.
class BatchNormalization(layers.Layer):
    def __init__(self, is_training=False, **kwargs):
        super(BatchNormalization, self).__init__()
        self.bn = layers.BatchNormalization(
            epsilon=1e-5, momentum=0.9, scale=True, trainable=is_training)

    def call(self, inputs, training):
        x = self.bn(inputs, training=training)
        return x
"""

# Define a function to create a generator.

In [13]:
def create_generator(input_shape, latent_dimension):
  generator_shape = (7, 7, 128)
  generator_size = np.prod(generator_shape)

  generator = models.Sequential(name='generator')

  generator.add(layers.Input(shape=(latent_dimension, ), name='input-layer'))

  generator.add(layers.Dense(units=generator_size, name='block-1-dense'))
  generator.add(layers.LeakyReLU(alpha=0.2, name='block-1-lrelu'))

  generator.add(layers.Reshape(generator_shape, name='block-1-2-reshape'))

  generator.add(layers.Conv2DTranspose(filters=128, kernel_size=(4, 4), strides=(2, 2), padding='same', name='block-2-deconv2d'))
  generator.add(layers.LeakyReLU(alpha=0.2, name='block-2-lrelu'))

  generator.add(layers.Conv2DTranspose(filters=128, kernel_size=(4, 4), strides=(2, 2), padding='same', name='block-3-deconv2d'))
  generator.add(layers.LeakyReLU(alpha=0.2, name='block-3-lrelu'))

  generator.add(layers.Conv2D(filters=1, kernel_size=(7, 7), padding='same', activation='tanh', name='fake-image'))

  return (generator)

# Create a generator using CNN and show corrosponding summary.

In [14]:
generator = create_generator(image_shape, latent_dimension)
generator.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ block-1-dense (Dense)                │ (None, 6272)                │         809,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block-1-lrelu (LeakyReLU)            │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block-1-2-reshape (Reshape)          │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block-2-deconv2d (Conv2DTranspose)   │ (None, 14, 14, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block-2-lrelu (LeakyReLU)            │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block-3-deconv2d (Conv2DTranspose)   │ (None, 28, 28, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block-3-lrelu (LeakyReLU)            │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fake-image (Conv2D)                  │ (None, 28, 28, 1)           │           6,273 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,339,905 (5.11 MB)

 Trainable params: 1,339,905 (5.11 MB)

 Non-trainable params: 0 (0.00 B)

# Define a function to create a discriminator.

In [15]:
def create_discriminator(input_shape):
  discriminator = models.Sequential(name='discriminator')

  discriminator.add(layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same', input_shape=input_shape, name='block-1-conv2d'))
  discriminator.add(layers.LeakyReLU(alpha=0.2, name='block-1-lrelu'))

  discriminator.add(layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2), padding='same', name='block-2-conv2d'))
  discriminator.add(layers.LeakyReLU(alpha=0.2, name='block-2-lrelu'))

  discriminator.add(layers.GlobalMaxPooling2D(name='gap'))
  discriminator.add(layers.Flatten(name='flatten'))

  discriminator.add(layers.Dense(units=1, name='prediction'))

  return (discriminator)

# Create a discriminator using CNN and show corrosponding summary.

In [16]:
discriminator = create_discriminator(image_shape)
discriminator.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ block-1-conv2d (Conv2D)              │ (None, 14, 14, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block-1-lrelu (LeakyReLU)            │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block-2-conv2d (Conv2D)              │ (None, 7, 7, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block-2-lrelu (LeakyReLU)            │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gap (GlobalMaxPooling2D)             │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ prediction (Dense)                   │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 74,625 (291.50 KB)

 Trainable params: 74,625 (291.50 KB)

 Non-trainable params: 0 (0.00 B)

# Define optimizers.

### Import required python modules.

In [17]:
from tensorflow.keras.optimizers import Adam

### Set an appropiate learning rate.

In [18]:
learning_rate = 0.0003

### Define a function to create an optimizer for a generator.

In [19]:
def create_generator_optimizer(learning_rate):
  optimizer = Adam(learning_rate=learning_rate, beta_1=0.5)
  return (optimizer)

### Create an optimizer for a generator.

In [20]:
generator_optimizer = create_generator_optimizer(learning_rate)

### Define a function to create an optimizer for a discriminator.

In [21]:
def create_discriminator_optimizer(learning_rate):
  optimizer = Adam(learning_rate=learning_rate, beta_1=0.5)
  return (optimizer)

### Create an optimizer for a discriminator.

In [22]:
discriminator_optimizer = create_discriminator_optimizer(learning_rate)

# Define a function to generate inputs for a generator.

In [23]:
def create_generator_inputs(input_batch, number_of_samples):
  generator_inputs = tf.random.normal([number_of_samples, latent_dimension])
  return (generator_inputs)

In [24]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Define a function to compute a generator loss.

In [25]:
def compute_generator_loss(fake_predictions):
  fake_labels = tf.zeros_like(fake_predictions)
  generator_loss = cross_entropy(fake_labels, fake_predictions)
  return (generator_loss)

# Define a function to update a generator.

In [26]:
def update_generator(input_batch):
  generator_inputs = create_generator_inputs(input_batch, batch_size)

  with tf.GradientTape() as tape:
    generated_images = generator(generator_inputs)
    fake_predictions = discriminator(generated_images)

    generator_loss = compute_generator_loss(fake_predictions)
    gradients = tape.gradient(generator_loss, generator.trainable_weights)
    generator_optimizer.apply_gradients(zip(gradients, generator.trainable_weights))

    return (generator_loss)

# Define a function to compute a discriminator loss.

In [27]:
def compute_discriminator_loss(real_predictions, fake_predictions):
  real_labels = tf.zeros_like(real_predictions)
  real_labels += 0.05 * tf.random.uniform(tf.shape(real_labels))
  real_loss = cross_entropy(real_labels, real_predictions)

  fake_labels = tf.ones_like(fake_predictions)
  fake_labels += 0.05 * tf.random.uniform(tf.shape(fake_labels))
  fake_loss = cross_entropy(fake_labels, fake_predictions)

  discriminator_loss = 0.5 * (real_loss + fake_loss)

  return (discriminator_loss)

# Define a function to update a discriminator.

In [28]:
 def update_discriminator(input_batch):
   real_images, real_labels = input_batch

   generator_inputs = create_generator_inputs(input_batch, batch_size)

   generated_images = generator(generator_inputs)

   with tf.GradientTape() as tape:
     real_predictions = discriminator(real_images)
     fake_predictions = discriminator(generated_images)

     discriminator_loss = compute_discriminator_loss(real_predictions, fake_predictions)
     gradients = tape.gradient(discriminator_loss, discriminator.trainable_weights)
     discriminator_optimizer.apply_gradients(zip(gradients, discriminator.trainable_weights))

     return (discriminator_loss)

# Define a function to train on an input batch.

In [29]:
def train_on_batch(input_batch):
  generator_loss = update_generator(input_batch)
  discriminator_loss = update_discriminator(input_batch)

  return {
            'generator': generator_loss,
            'discriminator': discriminator_loss
        }

# Define a function to decode a generated image.

In [30]:
def decode_image(input_image):
  input_image = input_image * 127.5 + 127.5
  return (input_image)

# Define a function to generate samples.

In [31]:
number_of_samples = 8

In [32]:
def generate_samples(generator_inputs):
  generated_images = generator.predict(generator_inputs)
  generated_images = generated_images.reshape(number_of_samples, image_shape[0], image_shape[1])

  generated_images = decode_image(generated_images)

  return (generated_images)

# Define a function to generate and show the generated images.

In [33]:
from google.colab.patches import cv2_imshow

def show_samples():
  generator_inputs = create_generator_inputs(None, number_of_samples)
  generated_images = generate_samples(generator_inputs)

  for index, image in enumerate(generated_images):
    cv2_imshow(image)

# Train DCGAN using a generator and a discriminator.

In [34]:
number_of_epochs = 10

In [ ]:
for current_epoch in range(number_of_epochs):
  print('epoch', str(current_epoch))

  for current_batch in train_dataset:
    current_losses = train_on_batch(current_batch)

  '''
  for key_value, loss_value in current_losses.items():
    print(key_value, '-', loss_value.numpy())
  '''

  show_samples()